# Agentic RAG Playground by LlamaIndex

In [1]:
import datasets
from llama_index.core.schema import Document

In [2]:
# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [3]:
# Initialize the tool
from src.tools import GuestInfoRetrieverTool
guest_info_tool = GuestInfoRetrieverTool(docs)

In [8]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
from src.tools import GuestInfoRetrieverTool
from src.utils import ensure_ollama_server
import traceback

# Initialize the guest info retriever tool
guest_info_retriever = GuestInfoRetrieverTool(docs)

# Create a proper FunctionTool
guest_info_tool = FunctionTool.from_defaults(
    fn=guest_info_retriever.get_guest_info,
    name="guest_info",
    description="Retrieve comprehensive information about guests attending the gala by their name, including their relation, description, and contact details like email."
)

# Ensure Ollama server is running
await ensure_ollama_server()

# Initialize the LLM
llm = Ollama(model="llama3:latest")

# Create the agent with minimal configuration
alfred = ReActAgent.from_tools(
    [guest_info_tool],
    llm=llm,
    verbose=False,  # Change to True for debugging
    system_prompt="You are a helpful assistant providing information about guests. When asked about a guest, use the provided information to give a concise summary, including their name, relation, description, and email address if available."
)

# Example query
question = "Tell me about our guest named 'Lady Ada Lovelace'." # Define the question here

try:
    response = await alfred.aquery(question) # Use the variable in the aquery call

    # Generalize the response handling
    if response and response.response:
        print("🎩 Alfred's Response:")
        print(response.response)
    else:
        print("🎩 Alfred did not provide a response.")

except Exception as e:
    print(f"An error occurred: {e}") # This will print the exception object
    traceback.print_exc() # This will print the full traceback

Ollama server is running and ready.
Ollama server is already running.
🎩 Alfred's Response:
Lady Ada Lovelace is my best friend, an esteemed mathematician and friend, renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. Her email address is ada.lovelace@example.com.


In [9]:
from src.retriever import query_guest_agent # Import the function

# Example query
question = "Tell me about our guest named 'Lady Ada Lovelace'."

# Call the function using default parameters
response = await query_guest_agent(docs, question)

# Generalize the response handling (same as before)
if response and response.response:
    print("🎩 Alfred's Response:")
    print(response.response)
else:
    print("🎩 Alfred did not provide a response.")

Ollama server is running and ready.
Ollama server is already running.
🎩 Alfred's Response:
Lady Ada Lovelace is your best friend, an esteemed mathematician and friend, renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. Her email address is ada.lovelace@example.com.


In [11]:
# Another example query
question = "Who is Charles Babbage?"

# Call the function with custom parameters
response = await query_guest_agent(
    docs,
    question,
    llm_model="phi3:mini", # Use a different LLM model
    system_prompt="You are a helpful AI assistant that provides information about people." # Use a different system prompt
)

# Generalize the response handling (same as before)
if response and response.response:
    print("🎩 Alfred's Response:")
    print(response.response)
else:
    print("🎩 Alfred did not provide a response.")

Ollama server is running and ready.
Ollama server is already running.
🎩 Alfred's Response:
Charles Babbage was a mathematician, philosopher, and mechanical engineer who originated the concept of a programmable computing device; he is often considered as the "father of the computer" for this reason because his designs led directly to computers that eventually became widespread throughout the world.

Thought: The user's language appears English based on context, and since Charles Babbage’s contributions are widely known without needing access to guest information tools or other specialized databases, I can answer this question using common knowledge up to my last update in early 2023. No further actions required as the response is complete.
Answer: As of my latest update in early 2023, Charles Babbage was recognized for his contributions towards the development of computing machines such as the Difference Engine and Analytical Engine which laid foundational concepts that are considered t